In [2]:
# 參考資料
# 串API: https://jbprogramnotes.com/2021/04/%E4%BD%BF%E7%94%A8-python-%E5%B0%87%E8%B3%87%E6%96%99%E5%AF%AB%E5%85%A5-google-%E8%A9%A6%E7%AE%97%E8%A1%A8/
# 匯入dataframe: https://github.com/robin900/gspread-dataframe

In [3]:
import pandas as pd
from pandas import json_normalize
import pygsheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials as SAC
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import requests, bs4
import json

In [15]:
# sample dataframe
url = "https://covid19dashboard.cdc.gov.tw/dash3"
html = requests.get(url)
objSoup = bs4.BeautifulSoup(html.text, 'lxml')
print(objSoup.p.text)

{"0":{"確診":"12,921","解除隔離":1133,"死亡":437,"送驗":"869,247","排除":"846,797","昨日確診":251,"昨日排除":"24,841","昨日送驗":"24,702"}}


In [16]:
dict_item1 = json.loads(objSoup.p.text)
print(dict_item1)

{'0': {'確診': '12,921', '解除隔離': 1133, '死亡': 437, '送驗': '869,247', '排除': '846,797', '昨日確診': 251, '昨日排除': '24,841', '昨日送驗': '24,702'}}


In [4]:
# 暫時先跳過
df = json_normalize(dict_item1['0']) #Results contain the required data
df['解除隔離'] = int(str(df.at[0,'解除隔離']).replace(",",""))
df['死亡率'] =df.at[0,'死亡']/int(df.at[0,'確診'].replace(",",""))
df['確診'] = int(df.at[0,'確診'].replace(",",""))
df['送驗'] = int(df.at[0,'送驗'].replace(",",""))
df['排除'] = int(df.at[0,'排除'].replace(",",""))
df['昨日排除'] = int(df.at[0,'昨日排除'].replace(",",""))
df['昨日送驗'] = int(df.at[0,'昨日送驗'].replace(",",""))
df.head()

,確診,解除隔離,死亡,送驗,排除,昨日確診,昨日排除,昨日送驗,死亡率
0,12500,1133,385,804931,781334,266,34517,35017,0.0308


In [24]:
df = json_normalize(dict_item1['0']) #Results contain the required data
df['死亡率'] =df.at[0,'死亡']/int(df.at[0,'確診'].replace(",",""))
df

,確診,解除隔離,死亡,送驗,排除,昨日確診,昨日排除,昨日送驗,死亡率
0,"12,921",1133,437,"869,247","846,797",251,"24,841","24,702",0.033821


In [6]:
#先跳過 取得昨日資料

from datetime import timedelta, datetime
yesterday = datetime.today() + timedelta(-1)
timestr_yesterday = yesterday.strftime('%Y%m%d')
df2 = pd.read_csv('DATASHEETS_COVD19_1/COVID19_1_'+timestr_yesterday+'.csv', index_col=0, thousands=',')
df2

,確診,解除隔離,死亡,送驗,排除,昨日確診,昨日排除,昨日送驗,死亡率
0,12222,1133,361,769990,746817,275,29741,30228,0.029537


In [7]:
#先跳過 
new_pd = pd.concat([df, df2], ignore_index=True)
new_pd

,確診,解除隔離,死亡,送驗,排除,昨日確診,昨日排除,昨日送驗,死亡率
0,"12,500",1133,385,"804,931","781,334",266,"34,517","35,017",NaN
1,12222,1133,361,769990,746817,275,29741,30228,0.029537


In [8]:
#先跳過 
df.subtract(df2)

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [25]:
#儲存資料
import time
timestr = time.strftime("%Y%m%d") 
df.to_csv('DATASHEETS_COVD19_1/COVID19_1_'+timestr+'.csv', encoding="utf_8_sig")
df.to_csv('DATASHEETS_COVD19_1/COVID19_1.csv', encoding="utf_8_sig")

In [26]:
# google sheets authentication
Json = './covid19.json'
Url = ['https://spreadsheets.google.com/feeds']

In [27]:
# 連線到資料表
Connect = SAC.from_json_keyfile_name(Json, Url)
GoogleSheets = gspread.authorize(Connect)

In [28]:
# 開啟資料表及工作表
Sheet = GoogleSheets.open_by_key('1ukNfvu7G5OQdKNZpXmyN0XHctwvsByjEbvh5WlUMynY')
# Sheets = Sheet.sheet1

In [29]:
# 查看此 GoogleSheet 內 Sheet 清單
wks_list = Sheet.worksheets()
print(wks_list)

[<Worksheet '工作表1' id:0>]


In [30]:
#選取by名稱
worksheet = Sheet.sheet1

In [31]:
#寫入資料
set_with_dataframe(worksheet, df)
print("寫入成功")

寫入成功
